# Vector Embedding using all-MiniLM-L6-v2 Hugging Face model which is light weight model


Author-Vansh Garg
vanshgarg25804@gmail.com

##Installation of Required libraries

In [1]:
pip install pandas numpy faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

##Embedding of Assessment description Scarped from SHL Webpage

Assessment description is the concatenation of Assessment Title, Assessment Desciption, Test Duration, Remote Testing, Adaptive,Test Type, Assessment for job level, and langugae of Assessment

For Embeeding Purposes Here We use the all-MiniLM-L6-v2 Hugging Face Model.

For Calculating the Similarity in between the different Embedding Here we Use the Cosine Similarity.

Save the Faiss index file and pkl files of metadata and texts

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# Load SHL catalog
catalog_df = pd.read_csv("/content/shl_assessments (5).csv")

# Initialize embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# model = SentenceTransformer("BAAI/bge-base-en-v1.5")

def build_text_representation(row):
    # Create a clean textual description for each test
    text=f"""
    {row['Title']}. {row['Description']}
    Duration: {row['Assessment Length']} | Remote: {row['Remote Testing']} | Adaptive: {row['Adaptive/IRT Support']}
    Type: {row['Test Type']} | Level: {row['Job Levels']} | Language: {row['Languages']}
    """
    # print(text)
    return text

# Generate descriptions
catalog_texts = catalog_df.apply(build_text_representation, axis=1).tolist()

# Generate normalized embeddings (for cosine similarity)
catalog_embeddings = model.encode(catalog_texts, normalize_embeddings=True)
catalog_embeddings = np.array(catalog_embeddings).astype("float32")

# Build FAISS index using inner product (which becomes cosine when vectors are normalized)
index = faiss.IndexFlatIP(catalog_embeddings.shape[1])
index.add(catalog_embeddings)

# Save
faiss.write_index(index, "catalog_index_cosine.faiss")
with open("catalog_texts.pkl", "wb") as f:
    pickle.dump(catalog_texts, f)
with open("catalog_metadata.pkl", "wb") as f:
    pickle.dump(catalog_df.to_dict(orient="records"), f)

print("✅ Cosine index built and saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Cosine index built and saved.


##Function for extracting text from url based on url is an linkedin job or any other link and return extracted text concatenated with user's prompt

In [6]:
# Utility to extract plain text from a URL (e.g. job description page)
def extract_text_from_url(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")

        # If it's a LinkedIn job URL
        if "linkedin.com" in url:
            containers = soup.find_all(class_=["jobs-description__container", "jobs-description__container--condensed"])
            text = " ".join([container.get_text(separator=' ') for container in containers])
        else:
            for tag in soup(["script", "style"]):
                tag.decompose()
            text = soup.get_text(separator=' ')

        return ' '.join(text.split())

    except Exception as e:
        print(f"[!] Error scraping URL: {e}")
        return ""

##Utility Function for Generating the Embedding of user's Query and find the most similar items from Saved Assessment






In [ ]:
def search_catalog_cosine(user_query, top_k=10):
    # Check for URLs in the query
    urls = re.findall(r'(https?://\S+)', user_query)
    if urls:
        print("URL detected in query. Scraping content...")
        for url in urls:
            scraped_text = extract_text_from_url(url)
            if scraped_text:
                user_query += f" Job Description: {scraped_text}"
                print(user_query)
    # Encode and normalize query
    query_embedding = model.encode([user_query], normalize_embeddings=True).astype("float32")

    # Search FAISS (inner product = cosine similarity for normalized vectors)
    scores, indices = index.search(query_embedding, k=top_k)

    results = []
    for idx, score in zip(indices[0], scores[0]):
        item = catalog_metadata[idx]
        item_with_score = item.copy()
        item_with_score["similarity"] = float(score)
        results.append(item_with_score)

    return results

##For Taking Query from user and convert them to json format

In [3]:
import pickle
import faiss
import numpy as np
import re
import requests
import json
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

# Load FAISS index (must be built with cosine similarity!)
index = faiss.read_index("catalog_index_cosine.faiss")

# Load catalog texts and metadata
with open("catalog_texts.pkl", "rb") as f:
    catalog_texts = pickle.load(f)

with open("catalog_metadata.pkl", "rb") as f:
    catalog_metadata = pickle.load(f)

# Load same model
model = SentenceTransformer("all-MiniLM-L6-v2")


# Example usage
if __name__ == "__main__":
    query = input("Enter your hiring query: ")
    results = search_catalog_cosine(query)

    output_data = []

    print("\nTop Recommendations (by Cosine Similarity):\n")
    for i, item in enumerate(results, 1):
        adaptive_support = "Yes" if item.get("Adaptive/IRT Support") == 1 else "No"

        result_object = {
            "Rank": i,
            "Title": item.get("Title", "N/A"),
            "URL": item.get("URL", "N/A"),
            "Duration": item.get("Assessment Length", "N/A"),
            "Remote Testing": item.get("Remote Testing", "N/A"),
            "Adaptive": adaptive_support,
            "Test Type": item.get("Test Type", "N/A"),
            "Job Levels": item.get("Job Levels", "N/A"),
            "Languages": item.get("Languages", "N/A"),
            "Cosine Similarity Score": item.get("similarity")
        }

        output_data.append(result_object)

        # Optional print for console
        print(f"{i}. {result_object['Title']}")
        print(f"   URL: {result_object['URL']}")
        print(f"   Duration: {result_object['Duration']} | Remote: {result_object['Remote Testing']} | Adaptive: {result_object['Adaptive']}")
        print(f"   Type: {result_object['Test Type']} | Level: {result_object['Job Levels']} | Language: {result_object['Languages']}")
        # print(f"   🔍 Cosine Similarity Score: {result_object['Cosine Similarity Score']:.4f}\n")

    # Save results to a JSON file
    with open("search_results.json", "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=4)

    print("\nResults saved to 'search_results.json'")


Enter your hiring query: I am hiring for Java developers who can also collaborate effectively with my business teams. Looking  for an assessment(s) that can be completed in 40 minutes. 

Top Recommendations (by Cosine Similarity):

1. Assessment and Development Center Exercises
   URL: https://www.shl.com/solutions/products/product-catalog/view/assessment-and-development-center-exercises/
   Duration: Approximate Completion Time in minutes = Variable | Remote: Yes | Adaptive: No
   Type: Assessment Exercises | Level: Director, Entry-Level, Executive, Front Line Manager, General Population, Graduate, Manager, Mid-Professional, Professional Individual Contributor, Supervisor, | Language: English International, English (USA),
2. Java 8 (New)
   URL: https://www.shl.com/solutions/products/product-catalog/view/java-8-new/
   Duration: Approximate Completion Time in minutes = 18 | Remote: Yes | Adaptive: No
   Type: Knowledge & Skills | Level: Mid-Professional, Professional Individual Contri

In [5]:
data=pd.read_csv("/content/shl_assessments (5).csv")
data.head(10)

,Title,Description,Job Levels,Languages,Assessment Length,Test Type,Remote Testing,Remote Testing (Yes),Adaptive/IRT Support,URL,Ability & Aptitude,Biodata & Situational Judgement,Competencies,Development & 360,Assessment Exercises,Knowledge & Skills,Personality & Behavior,Simulations
0,Global Skills Development Report,This report is designed to be given to individ...,"Director, Entry-Level, Executive, General Popu...",Not defined,Not defined,"Ability & Aptitude, Competencies, Personality ...",Yes,1,0,https://www.shl.com/solutions/products/product...,1,1,1,1,1,0,1,0
1,.NET Framework 4.5,The.NET Framework 4.5 test measures knowledge ...,"Professional Individual Contributor, Mid-Profe...","English (USA),",Approximate Completion Time in minutes = 30,Knowledge & Skills,Yes,1,1,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
2,.NET MVC (New),Multi-choice test that measures the knowledge ...,"Mid-Professional, Professional Individual Cont...","English (USA),",Approximate Completion Time in minutes = 17,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
3,.NET MVVM (New),Multi-choice test that measures the knowledge ...,"Mid-Professional, Professional Individual Cont...","English (USA),",Approximate Completion Time in minutes = 5,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
4,.NET WCF (New),Multi-choice test that measures the knowledge ...,"Mid-Professional, Professional Individual Cont...","English (USA),",Approximate Completion Time in minutes = 11,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
5,.NET WPF (New),Multi-choice test that measures the knowledge ...,"Mid-Professional, Professional Individual Cont...","English (USA),",Approximate Completion Time in minutes = 9,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
6,.NET XAML (New),Multi-choice test that measures the knowledge ...,"Mid-Professional, Professional Individual Cont...","English (USA),",Approximate Completion Time in minutes = 5,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
7,Accounts Payable (New),Multiple-choice test that measures the knowled...,"Entry-Level, Graduate, Mid-Professional, Profe...","English (USA),",Approximate Completion Time in minutes = 9,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0
8,Accounts Payable Simulation (New),Simulated data entry test that measures the ab...,"Entry-Level, Graduate, Mid-Professional, Profe...","English (USA),",Approximate Completion Time in minutes = 8,Simulations,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,0,0,1
9,Accounts Receivable (New),Multiple-choice test that measures the knowled...,"Entry-Level, Graduate, Mid-Professional, Profe...","English (USA),",Approximate Completion Time in minutes = 13,Knowledge & Skills,Yes,1,0,https://www.shl.com/solutions/products/product...,0,0,0,0,0,1,0,0


In [5]:
import pickle

# Assuming 'model' is your trained model object
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [7]:
model = SentenceTransformer("all-MiniLM-L6-v2")
model.save("my_local_model")


In [9]:
import os
print(os.listdir())


['.config', 'shl_assessments (5).csv', 'search_results.json', 'model.pkl', 'catalog_index_cosine.faiss', 'catalog_metadata.pkl', 'my_local_model', 'catalog_texts.pkl', 'sample_data']


In [11]:
from google.colab import files
!zip -r /content/my_local_model.zip /content/my_local_model
files.download("/content/my_local_model.zip")


  adding: content/my_local_model/ (stored 0%)
  adding: content/my_local_model/special_tokens_map.json (deflated 80%)
  adding: content/my_local_model/model.safetensors (deflated 9%)
  adding: content/my_local_model/tokenizer.json (deflated 71%)
  adding: content/my_local_model/1_Pooling/ (stored 0%)
  adding: content/my_local_model/1_Pooling/config.json (deflated 57%)
  adding: content/my_local_model/config.json (deflated 48%)
  adding: content/my_local_model/tokenizer_config.json (deflated 73%)
  adding: content/my_local_model/config_sentence_transformers.json (deflated 34%)
  adding: content/my_local_model/sentence_bert_config.json (deflated 4%)
  adding: content/my_local_model/vocab.txt (deflated 53%)
  adding: content/my_local_model/README.md (deflated 64%)
  adding: content/my_local_model/modules.json (deflated 62%)
  adding: content/my_local_model/2_Normalize/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>